# Laboratorio 2
### Data science - Lynette Garcia
Daniel Gomez 21429
Abner Garcia

In [6]:
import pandas as pd
from ydata_profiling import ProfileReport

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
gas = pd.read_csv('gas.csv')

In [4]:
gas.head()

,Fecha,Aceites lubricantes,Asfalto,Bunker,Ceras,Combustible turbo jet,Diesel bajo azufre,Diesel ultra bajo azufre,Gas licuado de petróleo,Gasolina de aviación,...,Naftas,Petcoke,Petróleo crudo,Solventes,Butano,Diesel alto azufre,MTBE,Orimulsión,Petróleo reconstituido,Total importación
0,ene/2001,"0,00","27.748,99","214.581,84","0,00","0,00","0,00","0,00","194.065,74","820,00",...,"0,00","0,00","0,00","0,00","0,00","566.101,99","8.402,00","0,00","715.344,00","2.312.639,05"
1,feb/2001,"0,00","7.503,57","294.609,00","0,00","0,00","0,00","0,00","170.703,38","3.054,00",...,"0,00","0,00","0,00","0,00","0,00","489.525,80","0,00","0,00","370.166,00","1.769.208,76"
2,mar/2001,"0,00","26.304,32","315.263,80","0,00","0,00","0,00","0,00","161.837,37","677,00",...,"0,00","0,00","0,00","0,00","0,00","575.559,68","0,00","0,00","360.530,00","1.945.770,26"
3,abr/2001,"0,00","7.885,89","205.653,00","0,00","0,00","0,00","0,00","163.048,64","3.399,00",...,"0,00","0,00","0,00","0,00","0,00","437.745,42","8.184,00","0,00","359.527,00","1.623.637,76"
4,may/2001,"0,00","8.443,16","278.371,30","0,00","0,00","0,00","0,00","171.518,86","585,00",...,"0,00","0,00","0,00","0,00","0,00","552.609,13","12.680,00","0,00","723.346,00","2.262.726,64"


In [5]:
gas.describe()

,Fecha,Aceites lubricantes,Asfalto,Bunker,Ceras,Combustible turbo jet,Diesel bajo azufre,Diesel ultra bajo azufre,Gas licuado de petróleo,Gasolina de aviación,...,Naftas,Petcoke,Petróleo crudo,Solventes,Butano,Diesel alto azufre,MTBE,Orimulsión,Petróleo reconstituido,Total importación
count,283,281,281,281,280,281,281,281,281,281,...,281,281,281,281,281,281,281,281,281,281
unique,283,66,270,281,61,99,78,47,281,165,...,43,149,28,66,24,205,12,14,21,281
top,ene/2001,"0,00","0,00","214.581,84","0,00","0,00","0,00","0,00","194.065,74","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","2.312.639,05"
freq,1,216,10,1,220,183,204,235,1,109,...,237,133,254,216,242,77,270,268,261,1


In [7]:
analysis = ProfileReport(gas)
analysis.to_file('gas_analysis.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 244.20it/s]
